In [135]:
import matplotlib.pyplot as plt
import numpy as np
import sympy 

In [136]:
def generate_trainingdata(m=25):
    return np.array([0,0])+0.25*np.random.randn(m,2)

def f(x, minibatch):
    # loss function sum_{w in training data} f(x,w)
    y=0; count=0
    for w in minibatch:
        z=x-w-1
        y=y+sympy.Min(36*(z[0]**2+z[1]**2), (z[0]+4)**2+(z[1]+10)**2)
    return y/count

In [137]:
train = generate_trainingdata()

In [138]:
def shuffle(data):
    np.random.shuffle(data)
    return data

In [139]:
x1, x2 = sympy.symbols('x1 x2', real=True)
w1, w2 = sympy.symbols('w1 w2', real=True)

z1 = x1 - w1 - 1
z2 = x2 - w2 - 1

# Loss function and differentiations
loss_fn = sympy.Min(36*(z[0]**2+z[1]**2), (z[0]+4)**2+(z[1]+10)**2)
diff_x1 = sympy.diff(loss_fn, x1)
diff_x2 = sympy.diff(loss_fn, x2)

print(diff_x1)
print(diff_x2)

(-68*w1 + 68*x1 - 68)*Heaviside(-34*(-w1 + x1 - 1)**2 + (-w1 + x1 + 3)**2 - 34*(-w2 + x2 - 1)**2 + (-w2 + x2 + 8)**2) + (-2*w1 + 2*x1 + 6)*Heaviside(34*(-w1 + x1 - 1)**2 - (-w1 + x1 + 3)**2 + 34*(-w2 + x2 - 1)**2 - (-w2 + x2 + 8)**2)
(-68*w2 + 68*x2 - 68)*Heaviside(-34*(-w1 + x1 - 1)**2 + (-w1 + x1 + 3)**2 - 34*(-w2 + x2 - 1)**2 + (-w2 + x2 + 8)**2) + (-2*w2 + 2*x2 + 16)*Heaviside(34*(-w1 + x1 - 1)**2 - (-w1 + x1 + 3)**2 + 34*(-w2 + x2 - 1)**2 - (-w2 + x2 + 8)**2)


In [140]:
fn = sympy.lambdify(((x1, x2), (w1, w2)), loss_fn, modules='numpy')

# Using value from sympy.diff in functions to substitute x and w values easily
def dfdx1(x, w):
    x1, x2, w1, w2 = x[0], x[1], w[0], w[1]
    return (-72*w1 + 72*x1 - 72)*sympy.Heaviside(-36*(-w1 + x1 - 1)**2 + (-w1 + x1 + 3)**2 - 36*(-w2 + x2 - 1)**2 + (-w2 + x2 + 9)**2) + (-2*w1 + 2*x1 + 6)*sympy.Heaviside(36*(-w1 + x1 - 1)**2 - (-w1 + x1 + 3)**2 + 36*(-w2 + x2 - 1)**2 - (-w2 + x2 + 9)**2)
def dfdx2(x, w):
    x1, x2, w1, w2 = x[0], x[1], w[0], w[1]
    return (-72*w2 + 72*x2 - 72)*sympy.Heaviside(-36*(-w1 + x1 - 1)**2 + (-w1 + x1 + 3)**2 - 36*(-w2 + x2 - 1)**2 + (-w2 + x2 + 9)**2) + (-2*w2 + 2*x2 + 18)*sympy.Heaviside(36*(-w1 + x1 - 1)**2 - (-w1 + x1 + 3)**2 + 36*(-w2 + x2 - 1)**2 - (-w2 + x2 + 9)**2)


In [141]:
def approx_deriv(batch_points, x, init_point, b=1):
    sum_val = 0
    if x == 1:
        for i in range(b):
            sum_val = sum_val + dfdx1(init_point, batch_points[i])
    elif x == 2:
        for i in range(b):
            sum_val = sum_val + dfdx2(init_point, batch_points[i])
    return sum_val / b

In [142]:
def mb_sgd_const(fn, X, b_size = 1, alpha = 0.1, iterations = 50):
    x1 = X[0]
    x2 = X[1]
    X1 = np.array([x1])
    X2 = np.array([x2])
    func_vals = np.array(fn((x1, x2), (0,0)))
    
    N = len(X)
    
    for i in range(iterations):
        data = shuffle(train)
        for b in np.arange(0, N, b_size):
            samples_index = np.arange(b, b + b_size)
            batch = train[samples_index]
            ad_x1 = approx_deriv(batch, 1, data, b_size)
            ad_x2 = approx_deriv(batch, 2, data, b_size)
            x1 = x1 - (alpha * ad_x1)
            x2 = x2 - (alpha * ad_x2)
        X1 = np.append(X1, [x1], axis = 0)
        X2 = np.append(X2, [x2], axis = 0)
        func_vals = np.append(func_vals, fn((x1,x2), batch))
    return (X1, X2, func_vals)

In [143]:
def gd_const(fn, X, alpha = 0.01, iterations = 50):
    x1 = X[0]
    x2 = X[1]
    X1 = np.array([x1])
    X2 = np.array([x2])
    func_vals = np.array(fn((x1, x2), (0,0)))
    
    for i in range(iterations):
        batch = train[0]
        alpha_x1 = alpha * approx_deriv(batch, 1, X)
        alpha_x2 = alpha * approx_deriv(batch, 2, X)
        
        x1 = x1 - alpha_x1
        x2 = x2 - alpha_x2
        
        X1 = np.append(X1, [x1], axis = 0)
        X2 = np.append(X2, [x2], axis = 0)
        func_vals = np.append(func_vals, fn((x1,x2), batch))
        
    return (X1, X2, func_vals)

In [144]:
def mb_sgd_polyak(fn, X, fs = 0, eps = 0.0001, b_size = 1, iterations = 50):
    x1 = X[0]
    x2 = X[1]
    X1 = np.array([x1])
    X2 = np.array([x2])
    func_vals = np.array(fn(x1, x2))
    
    N = len(X)
    
    for i in range(iterations):
        data = shuffle(train)
        for b in np.arange(0, N, b_size):
            samples_index = np.arange(b, b + b_size)
            batch = train[samples_index]
            ad_x1 = approx_deriv(batch, b_size, 1, data)
            ad_x2 = approx_deriv(batch, b_size, 2, data)
            alpha_x1 = (fn(x1, x2) - fs) / (ad_x1**2 + eps) 
            alpha_x2 = (fn(x1, x2) - fs) / (ad_x2**2 + eps) 
            x1 = x1 - (alpha_x1 * ad_x1)
            x2 = x2 - (alpha_x2 * ad_x2)
        X1 = np.append(X1, [x1], axis = 0)
        X2 = np.append(X2, [x2], axis = 0)
        func_vals = np.append(func_vals, fn((x1,x2), batch))
    return (X1, X2, func_vals)

In [145]:
def mb_sgd_rms(fn, X, beta = 0.98, eps = 0.0001, b_size = 1, alpha = 0.1, iterations = 50):
    x1 = X[0]
    x2 = X[1]
    X1 = np.array([x1])
    X2 = np.array([x2])
    func_vals = np.array(fn(x1, x2))
    alpha_next = alpha
    N = len(X)
    for i in range(iterations):
        data = shuffle(train)
        for b in np.arange(0, N, b_size):
            samples_index = np.arange(b, b + b_size)
            batch = train[samples_index]
            ad_x1 = approx_deriv(batch, b_size, 1, data)
            ad_x2 = approx_deriv(batch, b_size, 2, data)
            x1_next = x1 - alpha_next * ad_x1
            grad_sum = (beta * grad_sum) + (1 - beta) * ((ad_x1)**2)
          
            x2_next = x2 - alpha_next * ad_x2
            grad_sum = (beta * grad_sum) + (1 - beta) * ((ad_x2)**2)
            
            alpha_next = alpha / (sympy.sqrt(grad_sum) + eps)
        X1 = np.append(X1, [x1_next], axis = 0)
        X2 = np.append(X2, [x2_next], axis = 0)
        func_vals = np.append(func_vals, fn((x1,x2), batch))
    return (X1, X2, func_vals)

In [146]:
def mb_sgd_hb(fn, X, beta = 0.98, b_size = 1, alpha = 0.1, iterations = 50):
    x1 = X[0]
    x2 = X[1]
    X1 = np.array([x1])
    X2 = np.array([x2])
    func_vals = np.array(fn(x1, x2))
    N = len(X)
    for i in range(iterations):
        data = shuffle(train)
        for b in np.arange(0, N, b_size):
            samples_index = np.arange(b, b + b_size)
            batch = train[samples_index]
            ad_x1 = approx_deriv(batch, b_size, 1, data)
            ad_x2 = approx_deriv(batch, b_size, 2, data)
            
            mx1 = (beta * mx1) + (alpha * ad_x1)
            x1 = x1 - mx1  
          
            mx2 = (beta * mx2) + (alpha * ad_x2)
            x2 = x2 - mx2
            
        X1 = np.append(X1, [x1], axis = 0)
        X2 = np.append(X2, [x2], axis = 0)
        func_vals = np.append(func_vals, fn((x1,x2), batch))
    return (X1, X2, func_vals)

In [147]:
def mb_sgd_adam(fn, X, beta1 = 0.98, beta2 = 0.9, b_size = 1, alpha = 0.1, iterations = 50):
    x1 = X[0]
    x2 = X[1]
    mx1 = 0; vx1 = 0
    mx2 = 0; vx2 = 0
    t = 0
    eps = 1e-8
    X1 = np.array([x1])
    X2 = np.array([x2])
    func_vals = np.array(fn(x1, x2))
    N = len(X)
    for i in range(iterations):
        data = shuffle(train)
        for b in np.arange(0, N, b_size):
            samples_index = np.arange(b, b + b_size)
            batch = train[samples_index]
            ad_x1 = approx_deriv(batch, b_size, 1, data)
            ad_x2 = approx_deriv(batch, b_size, 2, data)
            t += 1
            
            mx1 = beta1 * mx1 + (1 - beta1) * ad_x1
            vx1 = beta2 * vx1 + (1 - beta2) * (ad_x1**2)
            m_hat = mx1 / (1 - beta1 ** t)
            v_hat = vx1 / (1 - beta2 ** t)
            
            x1 = x1 - alpha * m_hat / (sympy.sqrt(v_hat) + eps)  
          
            mx2 = beta1 * mx2 + (1 - beta1) * ad_x2
            vx2 = beta2 * vx2 + (1 - beta2) * (ad_x2**2)
            m_hat = mx2 / (1 - beta1 ** t)
            v_hat = vx2 / (1 - beta2 ** t)
            
            x2 = x2 - alpha * m_hat / (sympy.sqrt(v_hat) + eps)
            
        X1 = np.append(X1, [x1], axis = 0)
        X2 = np.append(X2, [x2], axis = 0)
        func_vals = np.append(func_vals, fn((x1,x2), batch))
    return (X1, X2, func_vals)